In [1]:
import sys
sys.path.extend(["../src", "/Users/vasudevgupta/local/transformers/src"])

In [2]:
from wav2vec2 import Wav2Vec2Processer, Wav2Vec2ForCTC
import tensorflow as tf

In [3]:
processor = Wav2Vec2Processer(is_tokenizer=False)
tokenizer = Wav2Vec2Processer(is_tokenizer=True)
model = Wav2Vec2ForCTC.from_pretrained("../wav2vec2-base-960h")

Loading weights locally from `../wav2vec2-base-960h`
Total number of loaded variables: 212


In [21]:
batch, _ = tf.audio.decode_wav(tf.io.read_file("../data/sample.wav"))
batch = tf.transpose(batch, perm=(1, 0))
batch = tf.concat([batch, tf.random.normal(batch.shape)], axis=0)
batch = processor(batch)

batch

<tf.Tensor: shape=(2, 46797), dtype=float32, numpy=
array([[ 0.0042721 , -0.00247198,  0.00764414, ..., -0.00247198,
        -0.01596014, -0.02607627],
       [-1.1425714 ,  1.2993361 ,  0.53832394, ..., -1.5862964 ,
        -0.28949928, -1.0421019 ]], dtype=float32)>

In [22]:
output = model(batch)

In [23]:
output["logits"].shape

TensorShape([2, 145, 32])

In [24]:
tf_out = tf.argmax(output["logits"], axis=-1)
tf_out = tf.squeeze(tf_out).numpy()
tokenizer.decode(tf_out[0])

'SHE HAD YOUR DUCK SUP AND GREASY WASHWATER AL YEAR'

In [11]:
from transformers import Wav2Vec2ForCTC as HFWav2Vec2ForCTC
MODEL_ID = "facebook/wav2vec2-base-960h"
hf_model = HFWav2Vec2ForCTC.from_pretrained(MODEL_ID)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
from transformers import Wav2Vec2FeatureExtractor, Wav2Vec2CTCTokenizer
from transformers import Wav2Vec2Processor as HFWav2Vec2Processer
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(MODEL_ID)
hf_tokenizer =  Wav2Vec2CTCTokenizer.from_pretrained(MODEL_ID)

hf_processor = HFWav2Vec2Processer(feature_extractor=feature_extractor, tokenizer=hf_tokenizer)

In [13]:
import torch
sample, _ = tf.audio.decode_wav(tf.io.read_file("../data/sample.wav"))
sample = tf.transpose(sample, perm=(1, 0))

hf_batch = torch.from_numpy(sample.numpy()).float()
hf_batch.shape

torch.Size([1, 46797])

In [14]:
hf_batch = hf_processor(hf_batch, return_tensors="pt")['input_values'].squeeze(1)
hf_batch.shape, hf_batch

It is strongly recommended to pass the ``sampling_rate`` argument to this function.Failing to do so can result in silent errors that might be hard to debug.


(torch.Size([1, 46797]),
 tensor([[ 0.0043, -0.0025,  0.0076,  ..., -0.0025, -0.0160, -0.0261]]))

In [15]:
import numpy as np
np.max(hf_batch.numpy() - batch.numpy())

10.320303

In [16]:
with torch.no_grad():
    hf_out = hf_model(hf_batch)
hf_out = torch.argmax(hf_out["logits"], dim=-1).numpy().squeeze()
hf_out.shape

(145,)

In [17]:
hf_out.shape, tf_out.shape

((145,), (2, 145))

In [18]:
tokenizer.decode(hf_out)

'SHE HAD YOUR DUCK SUP AND GREASY WASHWATER AL YEAR'

In [19]:
hf_tokenizer.decode(hf_out)

'SHE HAD YOUR DUCK SUP AND GREASY WASHWATER ALL YEAR'

In [26]:
[tokenizer.id_to_token_mapping[t] for t in hf_out.tolist()]

['<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 'S',
 'S',
 'H',
 '<pad>',
 'E',
 '<pad>',
 '<pad>',
 '|',
 '|',
 'H',
 '<pad>',
 '<pad>',
 'A',
 '<pad>',
 '<pad>',
 'D',
 '<pad>',
 '|',
 '|',
 'Y',
 '<pad>',
 '<pad>',
 'O',
 'U',
 '<pad>',
 'R',
 '|',
 '|',
 'D',
 '<pad>',
 '<pad>',
 '<pad>',
 'U',
 '<pad>',
 'C',
 '<pad>',
 'K',
 '<pad>',
 '<pad>',
 '|',
 '|',
 '<pad>',
 'S',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 'U',
 '<pad>',
 '<pad>',
 'P',
 '<pad>',
 '|',
 '|',
 'A',
 'N',
 '<pad>',
 'D',
 '|',
 '|',
 'G',
 '<pad>',
 'R',
 '<pad>',
 'E',
 '<pad>',
 '<pad>',
 'A',
 '<pad>',
 'S',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 'Y',
 '<pad>',
 '|',
 '|',
 'W',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 'A',
 '<pad>',
 '<pad>',
 'S',
 '<pad>',
 'H',
 '<pad>',
 '<pad>',
 '<pad>',
 'W',
 '<pad>',
 '<pad>',
 '<pad>',
 'A',
 '<pad>',
 '<pad>',
 'T',
 'E',
 'E',
 'R',
 '|',
 '|',
 '|',
 'A',
 'L',
 '<pad>',
 '<p

In [20]:
np.max(hf_out.squeeze() - tf_out)

26

In [19]:
from transformers import Wav2Vec2FeatureExtractor, Wav2Vec2CTCTokenizer, Wav2Vec2ForCTC as HFWav2Vec2ForCTC
MODEL_ID = "facebook/wav2vec2-base-960h"
model = HFWav2Vec2ForCTC.from_pretrained(MODEL_ID)
tokenizer = Wav2Vec2CTCTokenizer.from_pretrained(MODEL_ID)
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(MODEL_ID)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
from transformers import AutomaticSpeechRecognitionPipeline
agent = AutomaticSpeechRecognitionPipeline(feature_extractor, model=model, tokenizer=tokenizer)

In [21]:
import soundfile as sf
speech, _ = sf.read("../data/sample.wav")
agent(speech)

HMMMMMMMMMM
HIIIIII
{'input_values': tensor([[ 0.0043, -0.0025,  0.0076,  ..., -0.0025, -0.0160, -0.0261]])}
tensor([[[ 13.0021, -26.9057, -26.6217,  ...,  -6.4053,  -8.6990,  -7.6950],
         [ 13.0199, -26.8902, -26.6022,  ...,  -6.4001,  -8.7187,  -7.7267],
         [ 13.0194, -26.8902, -26.6021,  ...,  -6.4002,  -8.7183,  -7.7272],
         ...,
         [ 13.0190, -26.8911, -26.6032,  ...,  -6.4001,  -8.7187,  -7.7278],
         [ 13.0168, -26.8930, -26.6053,  ...,  -6.4000,  -8.7142,  -7.7307],
         [ 13.0183, -26.8919, -26.6039,  ...,  -6.3995,  -8.7190,  -7.7278]]],
       grad_fn=<AddBackward0>)
TOKENS
torch.Size([145]) tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 12, 12, 11,  0,  5,  0,  0,
         4,  4, 11,  0,  0,  7,  0,  0, 14,  0,  4,  4, 22,  0,  0,  8, 16,  0,
        13,  4,  4, 14,  0,  0,  0, 16,  0, 19,  0, 26,  0,  0,  4,  4,  0, 12,
         0,  0,  0,  0,  0, 16,  0,  0, 23,  0,  4,  4,  7,  9,  0, 14,  4,  4,
        21,  0, 13,  0,  5,  0,  0,  

{'text': 'SHE HAD YOUR DUCK SUP AND GREASY WASHWATER ALL YEAR'}

In [22]:
hf_out

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 12, 12, 11,  0,  5,  0,
        0,  4,  4, 11,  0,  0,  7,  0,  0, 14,  0,  4,  4, 22,  0,  0,  8,
       16,  0, 13,  4,  4, 14,  0,  0,  0, 16,  0, 19,  0, 26,  0,  0,  4,
        4,  0, 12,  0,  0,  0,  0,  0, 16,  0,  0, 23,  0,  4,  4,  7,  9,
        0, 14,  4,  4, 21,  0, 13,  0,  5,  0,  0,  7,  0, 12,  0,  0,  0,
        0,  0, 22,  0,  4,  4, 18,  0,  0,  0,  0,  7,  0,  0, 12,  0, 11,
        0,  0,  0, 18,  0,  0,  0,  7,  0,  0,  6,  5,  5, 13,  4,  4,  4,
        7, 15,  0,  0,  0, 15,  4,  4,  4,  4, 22,  0,  0,  0,  5,  7,  7,
       13,  0,  4,  4,  4,  4,  0,  0,  0])

In [33]:
processor(batch)

<tf.Tensor: shape=(2, 46797), dtype=float32, numpy=
array([[ 0.0042721 , -0.00247198,  0.00764414, ..., -0.00247198,
        -0.01596014, -0.02607627],
       [ 0.13727087, -0.41194385, -0.3056962 , ..., -1.6625165 ,
        -1.0761024 , -0.70854855]], dtype=float32)>

In [40]:
feature_extractor(batch.numpy().tolist())

It is strongly recommended to pass the ``sampling_rate`` argument to this function.Failing to do so can result in silent errors that might be hard to debug.


{'input_values': array([[ 0.0042721 , -0.00247198,  0.00764414, ..., -0.00247198,
        -0.01596014, -0.02607627],
       [ 0.13727087, -0.41194386, -0.3056962 , ..., -1.66251652,
        -1.07610243, -0.70854858]])}

In [36]:
batch, hf_batch

(<tf.Tensor: shape=(2, 46797), dtype=float32, numpy=
 array([[ 3.0517578e-05, -3.0517578e-05,  6.1035156e-05, ...,
         -3.0517578e-05, -1.5258789e-04, -2.4414062e-04],
        [ 1.4000420e-01, -4.0828085e-01, -3.0221304e-01, ...,
         -1.6567366e+00, -1.0713152e+00, -7.0438349e-01]], dtype=float32)>,
 tensor([[ 3.0518e-05, -3.0518e-05,  6.1035e-05,  ..., -3.0518e-05,
          -1.5259e-04, -2.4414e-04],
         [ 1.4000e-01, -4.0828e-01, -3.0221e-01,  ..., -1.6567e+00,
          -1.0713e+00, -7.0438e-01]]))